In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D, \
                            MaxPooling2D, BatchNormalization, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
#from keras_tqdm import TQDMNotebookCallback

from keras.callbacks import TensorBoard
from keras import backend as K

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions

from datetime import datetime
import os

import numpy as np
import pandas as pd

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'fc_model.h5'

weight_name = 'secend_try_vgg16.h5'

# dimensions of our images.
img_width, img_height = 200, 150

train_data_dir = 'data/data/train'
validation_data_dir = 'data/data/validation'
nb_train_samples = 1600
nb_validation_samples = 695
epochs = 10#50

batch_size = 16

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Using TensorFlow backend.


Found 1600 images belonging to 2 classes.
Found 695 images belonging to 2 classes.


In [3]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_height, img_width)
else:
    input_shape = (img_height, img_width, 3)
    
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu', name='block1_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', name='block2_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

# Add Fully Connected Layer
model.add(Flatten())
model.add(Dense(256, activation='relu', name='dense_1_lld'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu', name='dense_1_2ld'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:10]:
    layer.trainable = False

In [ ]:
model.load_weights('secend_try_vgg16_ld.h5', by_name=True)

x = 1
if x:
    optimizer='adam'
else:
    optimizer=SGD(lr=0.0001, momentum=0.9, nesterov=True)

    
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])



epochs = 20

import keras
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto')
save_model = keras.callbacks.ModelCheckpoint('weights_VGG_LD.{epoch:02d}-{val_loss:.2f}.hdf5', save_best_only=True)
TensorBoard = keras.callbacks.TensorBoard(log_dir='./logs_VGG_LD', batch_size=batch_size)


history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose = 2,
    callbacks=[save_model, TensorBoard])
               
model.save_weights('secend_try_vgg16_ld.h5','w')

Epoch 1/20
 23/100 [=====>........................] - ETA: 19814s - loss: 1.0557 - acc: 0.6440

In [ ]:
model.save_weights('secend_try_vgg16_lbd.h5','w')